1. Привести в туториале пример группировки и квантизации
3. AB-test сделать 
    - разобраться в пайплайне и сделать его
    - вставить info_col
    - добавить метрики cuped и did
2. написать тесты
3. как работать с пропусками в АА (ttest)
- название переменных поправить

**Done**
- протестить один и несколько таргетов
- добавить info_col (iterable или str) в AAtest

# How to perform AA or AB tests

## 0. Import Libraries

In [100]:
import pandas as pd
import numpy as np
from lightautoml.addons.hypex.ABTesting.ab_tester import AATest#, ABTest
from lightautoml.addons.hypex.utils.tutorial_data_creation import create_test_data

np.random.seed(52) #needed to create example data

## 1. Create or upload your dataset
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 

In [101]:
data = create_test_data(rs=52)
data

No NaN added


,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
0,0,0,0,488.0,414.444444,67,M,E-commerce
1,3,0,0,501.5,424.333333,31,M,Logistics
2,10,0,0,522.5,416.222222,64,M,E-commerce
3,12,0,0,472.0,423.777778,43,M,E-commerce
4,13,0,0,508.5,424.222222,36,F,E-commerce
...,...,...,...,...,...,...,...,...
5365,9991,0,0,482.5,421.888889,23,F,E-commerce
5366,9992,0,0,491.5,424.000000,44,M,E-commerce
5367,9994,0,0,486.0,423.777778,27,F,Logistics
5368,9996,0,0,500.5,430.888889,56,F,E-commerce


## 2. AATest

### 2.0 Initialize parameters
info_col used to define informative attributes that should NOT be part of testing, such as user_id and signup_month <br>

In [96]:
info_cols = ['user_id', 'signup_month']
target = 'post_spends'

### 2.1 Simple AA-test
This is the easiest way to initialize and calculate metrics on a AA-test on 10 iterations<br>
Use it when you are clear about each attribute or if you don't have any additional task conditions (like grouping)

In [97]:
experiment = AATest(data=data, info_cols=info_cols, target_fields=target)

In [98]:
experiment_result, dict_of_datas = experiment.search_dist_uniform_sampling()
experiment_result.head(3)

  0%|          | 0/10 [00:00<?, ?it/s]

,random_state,post_spends a mean,post_spends b mean,post_spends ab delta %,post_spends t_test p_value,post_spends ks_test p_value,post_spends t_test passed,post_spends ks_test passed,mean_tests_score
0,0,427.848003,427.848003,0.0,1.0,1.0,True,True,1.0
1,1,427.672046,427.672046,0.0,1.0,1.0,True,True,1.0
2,2,428.380095,428.380095,0.0,1.0,1.0,True,True,1.0


In [99]:
dict_of_datas[0]

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,group
0,3,0,0,501.5,424.333333,31,M,Logistics,test
1,10,0,0,522.5,416.222222,64,M,E-commerce,test
2,12,0,0,472.0,423.777778,43,M,E-commerce,test
3,13,0,0,508.5,424.222222,36,F,E-commerce,test
4,14,0,0,497.0,421.777778,26,M,Logistics,test
...,...,...,...,...,...,...,...,...,...
5365,9987,0,0,467.0,431.555556,48,M,Logistics,control
5366,9988,0,0,501.5,423.222222,55,F,Logistics,control
5367,9991,0,0,482.5,421.888889,23,F,E-commerce,control
5368,9992,0,0,491.5,424.000000,44,M,E-commerce,control


In [64]:
if all(data.columns) == all(dict_of_datas[0].drop(columns=['group']).columns):
    print('a')
else:
    print('b')

a


#### 2.1.0 Single experiment
To perform single experiment you can use sampling_metrics()

In [50]:
random_state = 11

In [51]:
res = experiment.sampling_metrics(random_state=random_state)
metrics, dict_of_datas  = res['metrics'], res['data_from_experiment']
metrics

{'random_state': 11,
 'post_spends a mean': 427.78932340161515,
 'post_spends b mean': 427.78932340161515,
 'post_spends ab delta %': 0.0,
 'post_spends t_test p_value': 1.0,
 'post_spends ks_test p_value': 1.0,
 'post_spends t_test passed': True,
 'post_spends ks_test passed': True,
 'mean_tests_score': 1.0}

In [52]:
dict_of_datas[random_state]

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry,group
0,3,0,0,501.5,424.333333,31,M,Logistics,test
1,14,0,0,497.0,421.777778,26,M,Logistics,test
2,21,0,0,489.0,433.111111,30,M,E-commerce,test
3,28,3,1,479.5,527.888889,20,F,E-commerce,test
4,29,0,0,505.0,414.333333,30,M,E-commerce,test
...,...,...,...,...,...,...,...,...,...
5365,9988,0,0,501.5,423.222222,55,F,Logistics,control
5366,9990,0,0,490.0,426.000000,18,M,E-commerce,control
5367,9991,0,0,482.5,421.888889,23,F,E-commerce,control
5368,9992,0,0,491.5,424.000000,44,M,E-commerce,control


### 2.2 AA-test with grouping

In [32]:
info_cols = ['user_id', 'signup_month']
target = 'post_spends'

group_cols = 'industry'

In [33]:
experiment = AATest(data=data, info_cols=info_cols, target_fields=target, group_cols=group_cols)

In [34]:
experiment_result, dict_of_datas = experiment.search_dist_uniform_sampling()
experiment_result.head(3)

  0%|          | 0/10 [00:00<?, ?it/s]

,random_state,post_spends a mean,post_spends b mean,post_spends ab delta %,post_spends t_test p_value,post_spends ks_test p_value,post_spends t_test passed,post_spends ks_test passed,mean_tests_score
0,0,428.882431,428.882431,0.0,1.0,1.0,True,True,1.0
1,1,428.602956,428.602956,0.0,1.0,1.0,True,True,1.0
2,2,428.846995,428.846995,0.0,1.0,1.0,True,True,1.0


### 2.3 AA-testing with quantifying

In [35]:
info_cols = ['user_id', 'signup_month']
target = 'post_spends'

group_cols = 'industry'
quant_field = 'gender'

In [36]:
experiment = AATest(data=data, info_cols=info_cols, target_fields=target, group_cols=group_cols, quant_field=quant_field)

In [37]:
experiment_result, dict_of_datas = experiment.search_dist_uniform_sampling()
experiment_result.head(3)

  0%|          | 0/10 [00:00<?, ?it/s]

,random_state,post_spends a mean,post_spends b mean,post_spends ab delta %,post_spends t_test p_value,post_spends ks_test p_value,post_spends t_test passed,post_spends ks_test passed,mean_tests_score
0,0,428.347612,428.347612,0.0,1.0,1.0,True,True,1.0
1,1,427.967721,427.967721,0.0,1.0,1.0,True,True,1.0
2,2,428.347612,428.347612,0.0,1.0,1.0,True,True,1.0


# Metrics

## Data

In [244]:
prep = AATest(data=dict_of_datas[0], target_fields='pre_spends', info_cols=info_cols, group_cols='group')
data = prep.data
data

,pre_spends,post_spends,age,gender_F,gender_M,industry_E-commerce,industry_Logistics,group
0,501.5,424.333333,31,0,1,0,1,test
1,522.5,416.222222,64,0,1,1,0,test
2,472.0,423.777778,43,0,1,1,0,test
3,508.5,424.222222,36,1,0,1,0,test
4,497.0,421.777778,26,0,1,0,1,test
...,...,...,...,...,...,...,...,...
5365,467.0,431.555556,48,0,1,0,1,control
5366,501.5,423.222222,55,1,0,0,1,control
5367,482.5,421.888889,23,1,0,1,0,control
5368,491.5,424.000000,44,0,1,1,0,control


## Diff in means

$$\widehat {ATE}^{simple} = \bar Y_{t=1, d=1} - \bar Y_{t=1, d=0}$$

In [300]:
def diff_in_means(data:pd.DataFrame, target_col: str, groups_col: str): 
    """Counts difference in means of test and conrol group in target.
    
    Args:
        data: input dataframe
        target_col: name of target column
        groups_col: name of column with separation on test and control group
                    should contain values ('test', 'control')
                    
    Returns:
        difm: differemnce in means between test and control groups
    
    """
    
    mean_test = np.mean(data[data[groups_col]=='test'][target_col]) 
    mean_control = np.mean(data[data[groups_col]=='control'][target_col])
    difm = mean_test - mean_control
    
    return difm

In [301]:
diff_in_means(data, 'post_spends', 'group')

0.0

## Cuped

$$\hat Y^{cuped}_{1} = \bar Y_1 - \theta \bar X + \theta \mathbb E [X]$$

$$
\begin{aligned}
\text{Var} \left( \hat Y^{cuped}_{1} \right) &= \text{Var} \left( \bar Y_1 - \theta \bar X + \theta \mathbb E [X] \right) = \newline
&= \text{Var} \left( Y_1 - \theta X \right) / n = \newline
&= \Big( \text{Var} (Y_1) + \theta^2 \text{Var} (X) - 2 \theta \text{Cov} (X,Y) \Big) / n
\end{aligned}$$

$$
\theta^* = \frac{\text{Cov} (X,Y)}{\text{Var} (X)}
$$

$$
\text{Var} \left( \hat Y^{cuped}_{1} \right) = \text{Var} (\bar Y) (1 - \rho^2), \text{where}\;\rho=corr(X,Y)
$$

$$
\begin{aligned} 
\widehat {ATE}^{cuped} &= \hat Y^{cuped}_{1} (D=1) - \hat Y^{cuped}_{1}(D=0) = \newline &= \big( \bar Y_1 - \theta \bar X + \theta \mathbb E [X] \ \big| \ D = 1 \big) - \big( \bar Y_1 - \theta \bar X + \theta \mathbb E [X] \ \big| \ D = 0 \big) = \newline &= \big( \bar Y_1 - \theta \bar X \ \big| \ D = 1 \big) - \big( \bar Y_1 - \theta \bar X \ \big| \ D = 0 \big) \end{aligned}
$$

$$
\hat Y_{cuped,1} = \bar Y_1 - \theta \bar X
$$

In [302]:
from collections import namedtuple
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

#### Absolute

In [303]:
import scipy.stats as ss
from typing import Union

def absolute_ttest(
    control: Union[pd.DataFrame, pd.Series], 
    test: Union[pd.DataFrame, pd.Series], 
    control_before: Union[pd.DataFrame, pd.Series], 
    test_before: Union[pd.DataFrame, pd.Series]
):
    """Counts CUPED (Controlled-Experiment using Pre-Experiment Data) in absolute values.
    
    Args:
        control: target data after pilot in control group
        test: target data after pilot in test group
        
    Returns:
        result: named tuple with pvalue, effect, ci_length, left_bound and right_bound
    """
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) \
    / (np.var(control_before) + np.var(test_before))

    control = control - theta * control_before
    test = test - theta * test_before
    
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = ss.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    result = ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

    return result

# https://github.com/DimaLunin/AB_lifehacks/blob/8fbbcc6a82440bdb593ac670994f1f8c7a5ef2f3/CUPED.ipynb#L57

In [304]:
absolute_ttest(
    data[data['group']=='control']['post_spends'],
    data[data['group']=='test']['post_spends'],
    data[data['group']=='control']['pre_spends'],
    data[data['group']=='test']['pre_spends']
)

ExperimentComparisonResults(pvalue=1.0, effect=0.0, ci_length=2.973441563457471, left_bound=-1.4867207817287356, right_bound=1.4867207817287353)

#### Relative

In [305]:
def relative_cuped(
    control: Union[pd.DataFrame, pd.Series], 
    test: Union[pd.DataFrame, pd.Series], 
    control_before: Union[pd.DataFrame, pd.Series], 
    test_before: Union[pd.DataFrame, pd.Series]
):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before

    mean_den = np.mean(control)
    mean_num = np.mean(test_cup) - np.mean(control_cup)
    var_mean_den  = np.var(control) / len(control)
    var_mean_num  = np.var(test_cup) / len(test_cup) + np.var(control_cup) / len(control_cup)

    cov = -np.cov(control_cup, control)[0, 1] / len(control)

    relative_mu = mean_num / mean_den
    relative_var = var_mean_num / (mean_den ** 2)  + var_mean_den * ((mean_num ** 2) / (mean_den ** 4))\
                - 2 * (mean_num / (mean_den ** 3)) * cov
    
    relative_distribution = ss.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [306]:
relative_cuped(
    data[data['group']=='control']['post_spends'],
    data[data['group']=='test']['post_spends'],
    data[data['group']=='control']['pre_spends'],
    data[data['group']=='test']['pre_spends']
)

ExperimentComparisonResults(pvalue=1.0, effect=0.0, ci_length=0.006949761458391225, left_bound=-0.0034748807291956124, right_bound=0.003474880729195612)

#### Diff in diff

$$\widehat {ATE}^{DiD} = \big( \bar Y_{t=1, d=1} - \bar Y_{t=1, d=0} \big) - \big( \bar Y_{t=0, d=1} - \bar Y_{t=0, d=0} \big)$$

In [307]:
def diff_in_diff(data, target_col, target_col_before, groups_col):
    """Counts Difference in Difference.
    
    Metric uses pre-post analisys and counts differece in means in data before and after pilot:
    ATE = (y_test_after - y_control_after) - (y_test_before - y_control_before)
    
    Args:
        data: input data
        target_col: column name of target after pilot
        target_col_before: column name of target before pilot
        groups_col: name of column with separation on test and control group
                    should contain values ('test', 'control')
                    
    Returns:
        did: value of difference in difference
    """
    mean_test = np.mean(data[data[groups_col]=='test'][target_col]) 
    mean_control = np.mean(data[data[groups_col]=='control'][target_col])
    
    mean_test_before = np.mean(data[data[groups_col]=='test'][target_col_before]) 
    mean_control_before = np.mean(data[data[groups_col]=='control'][target_col_before])
    did = (mean_test - mean_control) - (mean_test_before-mean_control_before)
    
    return did

In [308]:
diff_in_diff(data, 'post_spends', 'pre_spends', 'group')

0.0